# MapReduce Exercise #1
Take a look at the `ratings.csv` file. It contains the ratings given by each user to the movies he watched.

We want to know how many movies got a rating of 5, 4, 3, 2 and 1.

To solve this problem you should define your own MapReduce algorithm and implement it with the `MRJob` python library in you Databricks cluster.

Because this is a warm up exercise, you'll get a Notebook with incomplete pieces of code for you to fill.

#### Task 1 - Think about the algorithm
Write in pseudo-code a MR algorithm to find how many movies got each rating.

#### Task 2 - Fill the incomplete code
Even though you though about your own algorithm, because this is your first time writing a MR job, I'm going to help you by giving you the code below to get started.

In [2]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class RatingsBreakdown(MRJob):
    def steps(self): 
        return [MRStep(mapper=self.mapper_get_ratings, 
                       reducer=self.reducer_count_ratings)]

    def mapper_get_ratings(self, _, line): 
        (userID, movieID, rating, timestamp) = line.split(',')
        yield rating, 1
        
    def reducer_count_ratings(self, key, values):
        yield key, sum(values)

When you feel ready, run the job:

In [4]:
RatingsBreakdown(args=["/dbfs/FileStore/tables/ratings.csv", "-o", "/dbfs/FileStore/tables/ratings_breakdown1"]).execute()

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/PythonShell.root.20190701.074811.813929
job output is in /dbfs/FileStore/tables/ratings_breakdown1
Removing temp directory /tmp/PythonShell.root.20190701.074811.813929...

Take a look at the output files from the job:

In [6]:
display(dbutils.fs.ls("/FileStore/tables/ratings_breakdown1"))


path,name,size
dbfs:/FileStore/tables/ratings_breakdown1/part-00000,part-00000,44
dbfs:/FileStore/tables/ratings_breakdown1/part-00001,part-00001,23
dbfs:/FileStore/tables/ratings_breakdown1/part-00002,part-00002,12
dbfs:/FileStore/tables/ratings_breakdown1/part-00003,part-00003,12
dbfs:/FileStore/tables/ratings_breakdown1/part-00004,part-00004,11
dbfs:/FileStore/tables/ratings_breakdown1/part-00005,part-00005,12
dbfs:/FileStore/tables/ratings_breakdown1/part-00006,part-00006,11


As you can see, there are many `part-*` files. There's one for each reducer and together they contain the MR job's output.
Take a look at one of those files:

In [8]:
dbutils.fs.head("/FileStore/tables/ratings_breakdown1/part-00000")

Out[ 4 ]: '"0.5"\t1370\n"1.0"\t2811\n"1.5"\t1791\n"2.0"\t7551\n'

The file contains rows separated by '\n' and columns seaprated by '\t', where the first column in the ranking and the second one is the amount of movies with that ranking.
In the next step we'll load the data into a table to better visualize the output.

In [10]:
%sql
CREATE TABLE IF NOT EXISTS ratings_breakdown(rating STRING, count INT) USING CSV OPTIONS (path "/FileStore/tables/ratings_breakdown1", header "false", delimiter='\t');
SELECT * FROM ratings_breakdown;

rating,count
0.5,1370
1.0,2811
1.5,1791
2.0,7551
2.5,5550
3.0,20047
3.5,13136
4.0,26818
5.0,13211
4.5,8551


Try to understand the query above, which parameters where given to create the table and why?

## Extra Exercise
As you can see, our job returned a table that contained the ratings 0.5, 1.0, 1.5, etc.
In addition, one of the rows contains the word 'rating' with count '1'.

In this exercise we'd like to make a minor fix:
1. If a rating has a decimal, round it up. For example: 4.5 will rounded to 5.0
2. Remove the word 'rating' from the output.

Both fixes will need to be introduced into the MR Job.

In [13]:
from mrjob.job import MRJob
from mrjob.step import MRStep
import math

class RatingsBreakdown(MRJob):
    def steps(self): 
        return [MRStep(mapper=self.mapper_get_ratings, 
                       reducer=self.reducer_count_ratings)]

    def mapper_get_ratings(self, _, line): 
        (userID, movieID, rating, timestamp) = line.split(',')
        if rating != 'rating':
          yield math.ceil(float(rating)), 1
        
    def reducer_count_ratings(self, key, values):
        yield key, sum(values)

In [14]:
RatingsBreakdown(args=["/dbfs/FileStore/tables/ratings.csv", "-o", "/dbfs/FileStore/tables/ratings_breakdown2"]).execute()

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/PythonShell.root.20190701.081235.182347
job output is in /dbfs/FileStore/tables/ratings_breakdown2
Removing temp directory /tmp/PythonShell.root.20190701.081235.182347...

In [15]:
%sql
CREATE TABLE IF NOT EXISTS ratings_breakdown2(rating STRING, count INT) USING CSV OPTIONS (path "/FileStore/tables/ratings_breakdown2", header "false", delimiter='\t');
SELECT * FROM ratings_breakdown2;

rating,count
1,4181
2,9342
3,25597
4,39954
5,21762
